In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from collections import Counter

In [2]:
train_data = pd.read_csv('/Users/jasminesamuel/Downloads/train (2).csv')
test_data = pd.read_csv('/Users/jasminesamuel/Downloads/test (2).csv')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Predict whether a given tweet is about a real distaster or not.

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [6]:
vocabulary = Counter()
for tweet in train_data['text']:
    #tweet = tf.strings.regex_replace(tweet, b'[^a-zA-Z]', b' ')
    tweet = tf.strings.split(tweet)
    vocabulary.update(list(tweet.numpy()))

In [7]:
len(vocabulary.keys())

31924

It looks like we have 31924 words in our dictionary. Lets take the top 10,000 of them and used them in our model.

In [8]:
vocab_size = 10000
truncated_vocabulary = [ word for word, count in vocabulary.most_common()[:vocab_size]]

In [9]:
#preprocess to replace each word with its ID (index)
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

In [10]:
table.lookup(tf.constant([b'this movie was faaaaaaantastic'.split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   24,   337,    21, 10770]])>

In [11]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [13]:
train_set = train_data[['text','target']]
target = train_set.pop('target')
target = target.values.reshape((7613, 1))
dataset = tf.data.Dataset.from_tensor_slices((train_set.values, target))

Lets make sure that the dataset converted correctly. We are going to iterate over the dataset and print it's features and target

In [14]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [b'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'], Target: [1]
Features: [b'Forest fire near La Ronge Sask. Canada'], Target: [1]
Features: [b"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"], Target: [1]
Features: [b'13,000 people receive #wildfires evacuation orders in California '], Target: [1]
Features: [b'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '], Target: [1]


In [15]:
train_set = dataset.map(encode_words).prefetch(1)

In [16]:
# This is broken
batch_size = 32
train_set = dataset.batch(batch_size).prefetch(1)
tweet_model = model.fit(train_set, epochs=5)

Epoch 1/5
7613/7613 [==============================] - 177s 23ms/step - loss: 0.6726 - accuracy: 0.5719
Epoch 2/5
7613/7613 [==============================] - 191s 25ms/step - loss: 0.6060 - accuracy: 0.6632
Epoch 3/5
7613/7613 [==============================] - 176s 23ms/step - loss: 0.5456 - accuracy: 0.7299
Epoch 4/5
7613/7613 [==============================] - 173s 23ms/step - loss: 0.5228 - accuracy: 0.7500
Epoch 5/5
7613/7613 [==============================] - 173s 23ms/step - loss: 0.5137 - accuracy: 0.7578


In [17]:
train_set


<PrefetchDataset shapes: ((1,), (1,)), types: (tf.int64, tf.int64)>